In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
from tqdm import tqdm

zf = zipfile.ZipFile('drive/MyDrive/DL_dataset/monkeys.zip')
for file in tqdm(zf.infolist()):
    zf.extract(file)

100%|██████████| 1371/1371 [00:08<00:00, 162.94it/s]


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

import numpy as np

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
from pytorchtools_st import EarlyStopping

In [ ]:
train_monkeys = ImageFolder('training/training', transform=transforms.Compose([transforms.Resize([224, 224]), 
                                                                               transforms.ToTensor(),
                                                                               transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]))

train_loader = DataLoader(train_monkeys, batch_size=32)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torchvision.models.vgg16(pretrained=True)
model = model.to(device = device)

model.classifier[6] = nn.Linear(4096, 10)
for param in model.parameters():
  param.requires_grad = False

for param in model.classifier.parameters(): # Обучаются только параметры линейных слоев
  param.requires_grad = True
 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

early_stop = EarlyStopping(delta = 0.5, patience = 3, verbose = False) # Ранняя остановка

n_epochs = 500

train_history_loss = list()
epochs_history = list()

for epoch in range(1, n_epochs + 1):
  epoch_loss = 0

  model.train() # явно указываем обучение
  for X_batch, y_batch in train_loader:
    X_batch, y_batch = X_batch.to(device='cuda'), y_batch.to(device='cuda')
    # forward
    out = model(X_batch)
    # loss
    batch_loss = criterion(out, y_batch)
    # backward
    batch_loss.backward()
    # optimization
    optimizer.step()
    optimizer.zero_grad()
    epoch_loss += batch_loss.item()

  train_history_loss.append(epoch_loss/len(train_loader))
  epochs_history.append(epoch)

  if epoch % 2 == 0:
    print(f'Epoch: [{epoch}/{n_epochs}], Train_loss: {train_history_loss[-1]}')

  early_stop(train_history_loss[-1], model)
  
  if early_stop.early_stop:
      print(f"Early stopping on epoch: [{epoch}/{n_epochs}]")
      break

torch.save(model, '/content/drive/MyDrive/DL_dataset/model_monkeys_vgg16.pth')

In [ ]:
test_monkeys = ImageFolder('training/training', transform=transforms.Compose([transforms.Resize([224, 224]),
                                                                               transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), 
                                                                               transforms.ToTensor()]))
test = DataLoader(test_monkeys, batch_size = 32, shuffle = True)

correct, total = 0, 0

model.eval()
for X_test, y_test for test:
  X_test, y_test = X_test.to(device = 'cuda'), y_test.to(device = 'cuda')
  out = model(X_test)

  _, predicted = torch.argmax(out, dim = 1)
  total += y_test.shape[0]
  correct += int((predicted == y_test).sum())

print(f'Accuracy for vgg16: {correct/total')